In [3]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_core.globals import set_llm_cache
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())
from langchain_core.prompts import ChatPromptTemplate



/Users/lucamehl/Downloads/entangl/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# system_template = "You are comparing an architectural specification document a building code document, and flagging possible code violations or issues in the specification document. List these issues below, using technical language."
system_template = "List any potential issues or building code violations that could occur when building the components described in the following list of architectural components (provided as JSON objects), using technical language. "

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{json_object}")]
)

chain = prompt_template | model
import json


In [7]:
from pprint import pprint as pp

filename = 'arch_rac_example_processed_with_descriptions'

with open('{}.json'.format(filename), 'r') as file:
    data = json.load(file)


pp(data["Walls"])



[{'Area': '22.35498965607848',
  'Base Constraint': 'Level(Name=Level 2, Elevation=3000)',
  'Category': 'Walls',
  'Comments': 'None',
  'Description Human-Readable': 'This wall type features a height of 3300 mm, '
                                'is composed of various materials including '
                                'plasterboard, wood, and insulated panels, has '
                                'a length of approximately 6202 mm, and is '
                                'categorized as room bounding within the '
                                'project.',
  'Family and Type': 'WallType',
  'IfcGUID': '3lLx0gNe59vvExhby0Bfew',
  'Length': '6201.999999999788',
  'Materials': ['Finishes - Interior - Plasterboard',
                'Wood - Stud Layer',
                'Structure - Timber Insulated Panel - Insulation',
                'Structure - Timber Insulated Panel - OSB',
                'Concrete, Sand/Cement Screed'],
  'Room Bounding': '1',
  'Top Constraint': 'Level(Name=R

In [8]:
response = chain.invoke({"json_object": data["Ceilings"]})
print(response.content)

When evaluating the architectural specification for the described grid ceilings, several potential issues and building code violations may arise:

1. **Height Offset Compliance**: The specified height offset of 2700mm must comply with local building codes regarding ceiling heights. If the minimum ceiling height requirements for habitable spaces are higher than this, it could result in a code violation.

2. **Room Bounding Compliance**: The first ceiling is marked as "Room Bounding" while the second is not. If the second ceiling is located in a space that requires it to be room bounding (e.g., a living room), failing to meet this requirement could lead to issues with fire safety codes, ventilation, or egress requirements.

3. **Material Specification**: The use of "Default" and "Finishes - Interior - Plasterboard" needs to comply with fire resistance ratings required for ceilings in specific occupancy classifications. If the plasterboard does not meet the required fire resistance rating

# TODO: Fine-tune an LLM trained on the building code in the given country.

### This saves us the need to upload thousands of large building code documents.